In [6]:
pip install imutils


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25855 sha256=5dd000c3aee62dd826100757c8912e0e13fab04f8a5cb958b7128f860b70121e
  Stored in directory: c:\users\91984\appdata\local\pip\cache\wheels\59\1b\52\0dea905f8278d5514dc4d0be5e251967f8681670cadd3dca89
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os


In [8]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [9]:
DIRECTORY = r"D:\Mask_Detection\dataset\data"
CATEGORIES = ["with_mask", "without_mask"]

In [10]:
print("[INFO] loading images...")

data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(224, 224))
    	image = img_to_array(image)
    	image = preprocess_input(image)

    	data.append(image)
    	labels.append(category)

[INFO] loading images...


C:\My_Apps\anaconda3\envs\myenv\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [11]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [12]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

In [13]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [14]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))


9406464/9406464 [==============================] - 5s 1us/step


In [15]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [16]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [19]:
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
print("[INFO] compiling model...")



[INFO] compiling model...


In [24]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Set your initial learning rate and the number of epochs
INIT_LR = 0.001
EPOCHS = 5

# Define the learning rate schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate=INIT_LR,
    decay_steps=10000,
    decay_rate=0.9
)

# Use the learning rate schedule in the optimizer
opt = Adam(learning_rate=lr_schedule)

# Rest of your code
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])


In [25]:
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

[INFO] training head...
Epoch 1/5
188/188 [==============================] - ETA: 0s - loss: 0.0575 - accuracy: 0.9807WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 47 batches). You may need to use the repeat() function when building your dataset.


188/188 [==============================] - 461s 2s/step - loss: 0.0575 - accuracy: 0.9807 - val_loss: 0.0551 - val_accuracy: 0.9848
Epoch 2/5
188/188 [==============================] - 343s 2s/step - loss: 0.0448 - accuracy: 0.9859
Epoch 3/5
188/188 [==============================] - 339s 2s/step - loss: 0.0430 - accuracy: 0.9855
Epoch 4/5
188/188 [==============================] - 334s 2s/step - loss: 0.0375 - accuracy: 0.9872
Epoch 5/5
188/188 [==============================] - 345s 2s/step - loss: 0.0329 - accuracy: 0.9885


In [26]:
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

[INFO] evaluating network...
48/48 [==============================] - 81s 2s/step
              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       745
without_mask       0.99      0.99      0.99       766

    accuracy                           0.99      1511
   macro avg       0.99      0.99      0.99      1511
weighted avg       0.99      0.99      0.99      1511



In [27]:
print("[INFO] saving mask detector model...")
model.save("D://Mask_Detection//mask_detector.model", save_format="h5")


[INFO] saving mask detector model...


C:\My_Apps\anaconda3\envs\myenv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
!pip install tensorflow
